## Analysis of model

Here I produce the plots that were necessary for this analysis. It takes quite some time to run but it shows the histogram of the error and also the zenith distribution. It also produces predicted values using the model that was best up till now. 

In [1]:
import numpy as np

In [2]:
import pandas as pd
import sys,os

In [3]:
def load_files(batch):
    images = []
    labels = []
    for i in batch:
        print('Loading File: ' + i)
        x = np.load(i,allow_pickle=True).item()
        keys = x.keys()
        for key in keys:
            images.append(x[key][0])
            labels.append(x[key][1])
    return np.array(images),np.array(labels)

def get_feature(labels,feature):
    feature_values = []
    for i in labels:
        feature_values.append(i[feature])
    feature_values = np.array(feature_values)
    return feature_values

def get_cos_values(zenith,azimuth):
    cos1 = []
    cos2 = []
    cos3 = []
    for i,j in zip(zenith,azimuth):
        cos1.append(np.sin(i) * np.cos(j))
        cos2.append(np.sin(i) * np.sin(j))
        cos3.append(np.cos(i))
    return np.array(cos1),np.array(cos2),np.array(cos3)

In [4]:
output_best = 'SWNN_1.h5'
file_path = '/fs/scratch/PAS1495/amedina/'
y = os.listdir(file_path+'processed_3D')
file_names = []

for i in y:
    file_names.append(file_path+'processed_3D/'+i)

file_names_batched = list(np.array_split(file_names,4))

In [5]:
images,labels = load_files(file_names_batched[4][0:400])

IndexError: list index out of range

In [ ]:
images1 = images[:,:,:,:,[0,7]]

zenith_values = get_feature(labels,1)
azimuth_values = get_feature(labels,2)

zenith_check = list(zip(images1,zenith_values,azimuth_values))
new_images = []
new_zenith_values = []
new_azimuth_values = []
for i in zenith_check:
    #if i[1] > 2:
    new_images.append(i[0])
    new_zenith_values.append(i[1])
    new_azimuth_values.append(i[2])

new_images = np.array(new_images)

cos1,cos2,cos3=get_cos_values(new_zenith_values,new_azimuth_values)

cos_values = np.array(list(zip(cos1,cos2,cos3)))

In [ ]:
from keras.models import load_model
import tensorflow as tf
from keras import backend as K

In [ ]:
def predict_images(model_name,images):
    model = load_model(model_name,custom_objects={'loss_space_angle':loss_space_angle})
    predicted_cos_values = model.predict(images)
    return predicted_cos_values

def loss_space_angle(y_true,y_pred):
    subtraction = tf.math.subtract(y_true,y_pred)
    y = tf.matrix_diag_part(K.dot(subtraction,K.transpose(subtraction)))
    loss = tf.math.reduce_mean(y)
    return loss

In [ ]:
cnn_model = load_model('/users/PAS1495/amedina/work/DNN_Project/script/Network/cnn_model_1.h5',custom_objects={'loss_space_angle':loss_space_angle})
model = load_model('/users/PAS1495/amedina/work/DNN_Project/script/Network/'+output_best,custom_objects={'cnn_model':cnn_model,'loss_space_angle':loss_space_angle})
cos_values_pred = model.predict(new_images)

In [ ]:
cos_values_1_1 = [(i*2.0 - 1) for i in list(zip(*cos_values_pred)[0])]
cos_values_1_2 = [ (i*2.0 -1) for i in list(zip(*cos_values_pred)[1])]
cos_values_1_3 = [(i*2.0-1) for i in list(zip(*cos_values_pred)[2])]

In [ ]:
print(cos_values)
cos_values_1=zip(*cos_values)[0]
cos_values_2=zip(*cos_values)[1]
cos_values_3=zip(*cos_values)[2]

In [ ]:
import matplotlib.pylab as plt
import matplotlib

def plot_value(figure_number,x,y):
    plt.figure(figure_number)
    plt.hist2d(x,y,
               bins=100,
               norm=matplotlib.colors.LogNorm())
    #plt.xscale('log')
    plt.colorbar()
    plt.xlabel('Energy (GeV)')
    plt.ylabel('Zenith Error (Rad)')

## Zenith Distribution

In [ ]:
plt.figure(5)
plt.hist(cos_values_1,bins=100,label='Predicted',edgecolor='blue', facecolor="None")
plt.hist(cos_values_1_1,bins=100,label='True',edgecolor='red', facecolor="None")
plt.xlabel('sin(zenith)cos(azimuth)')
plt.ylabel('Count')
plt.legend()

In [ ]:
plt.figure(5)
plt.hist(cos_values_2,bins=100,label='Predicted',edgecolor='blue', facecolor="None")
plt.hist(cos_values_1_2,bins=100,label='True',edgecolor='red', facecolor="None")
plt.xlabel('sin(zenith)sin(azimuth)')
plt.ylabel('Count')
plt.legend()

In [ ]:
plt.figure(5)
plt.hist(cos_values_3,bins=100,label='Predicted',edgecolor='blue', facecolor="None")
plt.hist(cos_values_1_3,bins=100,label='True',edgecolor='red', facecolor="None")
plt.xlabel('cos(zenith)')
plt.ylabel('Count')
plt.legend()

In [ ]:
cos_new = []
for i in cos_values_1_3:
    if i < -1.0:
        continue
    elif i > 1.0:
        continue
    else:
        cos_new.append(i)
zenith = np.arccos(cos_new)   

In [ ]:
plt.figure(5)
plt.hist(zenith,bins=1000,label='Predicted',edgecolor='blue', facecolor="None")
plt.hist(new_zenith_values,bins=1000,label='True',edgecolor='red', facecolor="None")
plt.xlabel('Zenith')
plt.ylabel('Count')
plt.legend()

In [ ]:
weird = []
for i in zip(zenith,new_zenith_values):
    if abs(i[1]-i[0]) > 0.5:
        #print(i)
        #print(i[1]-i[0])
        weird.append(i)
list1,list2 = zip(*weird)
list_converted = []
for i in list2:
    list_converted.append(i*180/np.pi)
plt.hist(list_converted,bins=1000)
plt.show()

In [ ]:
def azimuth(sincos,sinsin):
    values = []
    for i,j in zip(sinsin,sincos):
        if i > 0:
            if j > 0:
                values.append(np.arctan(i/j))
            if j < 0:
                values.append(-np.arctan(i/j)+np.pi)
        if i < 0:
            if j > 0:
                values.append(np.arctan(i/j)+2.0*np.pi)
            if j < 0:
                values.append(-np.arctan(i/j)+np.pi)
    return values

In [ ]:
azimuth_predicted = azimuth(list(cos_values_1_1),list(cos_values_1_2))

In [ ]:
plt.figure(5)
plt.hist(azimuth_predicted,bins=100,label='Predicted',edgecolor='blue', facecolor="None")
plt.hist(new_azimuth_values,bins=100,label='True',edgecolor='red', facecolor="None")
plt.xlabel('Azimuth')
plt.ylabel('Count')
plt.legend()

In [ ]:
from sklearn.metrics import mean_squared_error
a1=np.histogram(azimuth_predicted,bins=100)[0]
a2=np.histogram(azimuth_values,bins=100)[0]
mse = mean_squared_error(a1,a2)
print(mse**0.5)

In [ ]:
b1,bins=np.histogram(zenith,bins=100)
b2,bins2=np.histogram(new_zenith_values,bins=100)
mse_search = []
i = len(b1)-1
while i > -1:
    mse1 = mean_squared_error(b1[i:len(b1)],b2[i:len(b2)])
    mse_search.append(mse1)
    i-=1
plt.plot(bins[1:len(bins)],mse_search[::-1])
plt.show()
print(zip(bins[1:len(bins)],mse_search[::-1]))

In [ ]:
def space_angle_error(variable1,variable2):
    x = []
    for i,j in zip(variable1,variable2):
        magnitude1 = (i[0]**2.0+i[1]**2.0+i[2]**2.0)**0.5
        magnitude2 = (j[0]**2.0+j[1]**2.0+j[2]**2.0)**0.5
        dot_product = (i[0]*j[0]+i[1]*j[1]+i[2]*j[2])
        error = np.arccos(dot_product/(magnitude1*magnitude2))
        x.append(error)
    return x,magnitude1,magnitude2

In [ ]:
value1 = zip(cos_values_1,cos_values_2,cos_values_3)
value2 = zip(cos_values_1_1,cos_values_1_2,cos_values_1_3)
error,mag1,mag2 = space_angle_error(value1,value2)
print(mag1)

In [ ]:
plt.figure(5)
plt.hist(error,bins=100,edgecolor='blue', facecolor="None")
plt.xlabel('Space Angle Error')
plt.ylabel('Count')
plt.legend()

In [ ]:
mean = np.mean(error)
print(mean)
median = np.median(error)
print(median)
percentile = np.percentile(error,95)
print(percentile)
percentile2 = np.percentile(error,75)
print(percentile2)

In [ ]:
0.136*(180.0/np.pi)